# Кто выбирает президнетов

# Часть 2: Подготовка данных к построению тематической модели

In [1]:
import pandas as pd              # Пакет для работы с таблицами
import numpy as np               # Пакет для работы с векторами и матрицами

# Пакет для красивых циклов. При желании его можно отключить. 
# Тогда из всех циклов придётся  удалять команду tqdm_notebook.
from tqdm import tqdm_notebook   # подробнее: https://github.com/tqdm/tqdm

import re      # Пакет для работы с регулярными выражениями 
import pickle  # Пакет для подгрузки данных специфического для питона формата

# Токенизатор (дробит тест на отдельные слова)
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

# Лемматизатор (приводит слова к нормальной форме)
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

# Список стоп-слов
from nltk.corpus import stopwords
stop = stopwords.words('russian')

# Библиотека для распараллеливания кода
from joblib import Parallel, delayed

Подгрузим данные о каждом из кандидатов, приготовленные нами на предыдущем этапе и немного поколдуем над ними. 

In [2]:
# Путь к данным
path = 'data/'

# Фамилии кандидатов
candidates = ['Putin', 'Grudinin', 'Baburin', 'Titov',
              'Sobchak', 'Suraykin', 'Yavlinskiy', 'Zhirinovsky']

# для рисёрческих целей немного расширим список кандидатов тем-чьё-имя-нельзя-называть 
candidates += ['Navalny']

In [3]:
def dataLoad(candidat_name, path):
    """ 
        Возвращает чистую табличку 
        candidat_name: string
            фамилия кандидата
        path: string
            путь до директории с данными
    """
    # подгружаем таблицу для нашего кандидата
    with open(path + candidat_name + '_dataframe.pkl', 'rb') as f:
        df = pickle.load(f)
    return df

In [10]:
dataLoad('putin', path).iloc[20:25]

,date,copy_post_date,post_type,from_id,text,likes,reposts,comments,can_comment,views,copy_owner_id,type,platform,attachments,uid,wall_cnt
20,2017-10-03 17:21:37,None,post,203776471,,0,0,0,1,43,None,api,android,None,203776471,727
21,2017-10-09 23:31:44,None,post,234526594,Отправляю тебе ПРЕДСКАЗАНИЕ : \n\n [Счастливые...,0,0,0,1,None,None,api,None,"[page, link]",203776471,727
22,2017-10-10 06:04:00,None,post,203776471,,0,0,0,1,33,None,api,android,None,203776471,727
23,2017-10-13 22:38:46,None,post,203776471,,0,0,0,1,31,None,api,android,None,203776471,727
24,2017-10-15 05:03:54,None,post,203776471,,0,0,0,1,31,None,api,android,None,203776471,727


## 1. Подтаблица с текстами

In [21]:
df_text = pd.DataFrame( )
for can in candidates:
    # Идём по всем кандидатам, подгружаем таблицы
    df = dataLoad(can, path)
    # Соединяем весь текст, соответствующий каждому id в одно большое целое
    df_grby = df.groupby('uid')['text'].agg({'text' : lambda x: ' '.join(x)})
    df_grby['candidat_name'] = can  # Создаём дополнительный столбец с фамилией кандидата
    # Записываем таблицу, полученную по текущему кандидату в вектор
    df_text = df_text.append(df_grby, ignore_index=True)

df_text.head() # Посмотрим на то как это выглядит

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


,text,candidat_name
0,"Друзья, встречаемся сегодня в 19:00 У ТРК ""ЛОТ...",Putin
1,Вот это вещь! Я собрал: Осадная мортира! Ура! ...,Putin
2,Конкурс на 15 Рандомных ключей в steam.<br><br...,Putin
3,"И даже не смей думать, что ты можешь не выдерж...",Putin
4,"Художник Ринат Волигамси <br>""Большая Медведиц...",Putin


In [22]:
# Сохраняем табличку
df_text.to_csv(path + 'data_text.csv', sep=';')

В нашем распоряжении оказалась табличка с текстами, которую мы и будем использовать для тематического моделирования. Перед тем как использовать её, нам придётся почистить тексты от всякого мусора. 

* Приведём все слова к нижнему регистру, чтобы слово `Политик` и `политик` не считались отдельными словами.
* Очистим все тексты от html-тэгов и ссылок.
* Разрубим все тексты на отдельные слова. Такую разрубку называют __токенизацией__. 

Обычно, при работе с текстами руководствуются гипотезой, которая называется **мешком слов.** Смысл этой гипотезы состоит в том, что порядок слов в предложении никак не сказывается на его смысле. 

* Порядок слов неважен
* Неважен порядок слов
* Слов порядок неважен 
* Порядок неважен слов 
* Слов неважен порядок
* Неважен слов порядок 

Конечно же с этой гипотезой можно поспорить и для этого вам придётся схлеснуться в словесной дуэли с одним известным персонажем. 


<img align="center" src="pictures/master.jpg" width="500"> 


Главным аргументом с вашей стороны, скорее всего, будет частица не, положение которой в предложении может координально влиять на смысл фразы. Да, порядок слов иногда важен. Тем не менее при моделировании мы должны чем-то жертвовать. Справедливости ради стоит отметить, что существует ряд более сложных моделей, которые учитывают порядок слов, но мы не будем их касаться в этом исследовании. Тем более, интуиция подсказывает, что пренебрежение порядком слов не должно никак сказаться на выделении тем. 

Грубо говоря, каждое слово это регрессор. Перед каждым словом надо оценивать коэффициент. То, с какой частотой оно наблюдается в статье - наше наблюдение. Наблюдений в сравнении с числом параметров, которое нужно будет оценить, очень мало. Постараемся уменьшить число регрессоров, выкинув не очень важные слова. 

Например, в текстах часто встречаются предлоги. Они не несут никакой смысловой информации о сути текста и есть почти в каждой группе. Найдётся огромное количество подобной лексики, которая для нашего набора текстов встречается повсеместно и не несёт никакой инормации. Обычно такие частые слова называют **стоп-словами.** Они путают нашу модель, перетягивая на себя внимание. От них необходимо избавиться.

Другим естественным путём для уменьшения числа регрессоров является приведение всех слов к одной форме. Всех существительных к именительному падежу и единственному числу, всех глаголов к инфинитиву и т.д. Такая обработка называется **лемматизацией.** Она делается по специальным словарям, составленным для языка. К счастью, для русского языка такой словарь реализован в ультракрутом пакете [`pymorthy2`.](http://pymorphy2.readthedocs.io/en/latest/)

Этот пакет превратит слово люди в слово человек, слова красивая и красивые в слово красивый, слова сделал и делал в делать. Таким образом количество различных слов в словаре здорово упадёт, а смысл не пострадает. 

Альтернативой лемматизации является **стемминг.** В ходе стемминга происходит обрубка приставок, cуффиксов и окончаний. Метод пытается выделить у каждого слова корневую основу. Для английского языка стемминг работает довольно хорошо. Для русского языка хорошо работает лемматизация. Подумайте на досуге о том почему. 

In [13]:
def htmlStrip(text):
    """ 
        Возвращает текст, очещенный от html тэгов
        text: string
            Текст поста 
    """
    
    return re.sub('<[^<]+?>', '', str(text)) 


def tokenLemma(text, stop = stop):
    """ 
        Возвращает список слов, очищенный от тэгов, пролемматизированный и без стоп слов
        text: string
            Текст поста
        
        parameters: list 
            stop: список из стоп-слов, example: ['политик', 'выбирать']
    """
    
    # понижение регистра, очистка от тэгов, токенизация
    out_1 = tokenizer.tokenize(htmlStrip(text.lower()))
    # лемматизация и очистка от стоп-слов
    out_2 = [morph.parse(tx)[0].normal_form for tx in out_1 if tx not in stop]
    return out_2

Предобработаем все посты из датафрейма. Немного распараллелим лемматизацию. Если это не сделать, то придётся ждать около суток. Если сделать, то поменьше (около 3-х часов).

In [ ]:
n_jobs = -1 # параллелим на все ядра 
result = Parallel(n_jobs=n_jobs)(delayed(tokenLemma)(
    text) for text in tqdm_notebook(df_text.text))

Отлично! Не прошло и суток, лемматизированный контент подписчиков оказался в наших цепких рисёрчерских ручках. Сохраняем лемматизированный текст как новую колонку нашего датафрейма и переходим к оцениванию модели. 

In [ ]:
df_lemm_text = df_text.drop('text', axis=1)
df_lemm_text['lemm_text'] = result

In [24]:
df_lemm_text.head()

,candidat_name,lemm_text
uid,,
145578958,Putin,"[друг, встречаться, сегодня, 19, 00, трк, лото..."
145579153,Putin,"[это, вещь, собрать, осадный, мортира, ура, до..."
145580272,Putin,"[конкурс, 15, рандомный, ключ, steam, сделать,..."
145580416,Putin,"[сметь, думать, мочь, выдержать, эрнест, хемин..."
145581817,Putin,"[художник, ринат, волигамси, большой, медведиц..."


In [20]:
# Сохраняем лемматизированную табличку
df_lemm_text.to_csv(path + 'data_text_lemm.csv', sep=';')

С предобработкой текстов мы закончили. Теперь объединим все маленькие таблички с кандидатами в одну огромную таблицу. 

## 2. Большая таблица с постами

In [25]:
df_all_posts = pd.DataFrame( )
for can in candidates:
    # Идём по всем кандидатам, подгружаем таблицы
    df = dataLoad(can, path)
    df['candidat_name'] = can
    # соединяем таблички в одну большую
    df_all_posts = df_all_posts.append(df, ignore_index=True)

print('Итоговая размрность таблицы:', df_all_posts.shape)
df_all_posts.head()

Итоговая размрность таблицы: (5150408, 17)


,date,copy_post_date,post_type,from_id,text,likes,reposts,comments,can_comment,views,copy_owner_id,type,platform,attachments,uid,wall_cnt,candidat_name
0,2017-04-30 21:19:36,None,post,203776389,,53,0,0,1,335,None,api,android,[photo],203776389,6,Putin
1,2017-05-01 10:21:07,None,post,203776389,,17,0,1,1,645,None,api,android,"[photo, photo]",203776389,6,Putin
2,2017-08-14 18:45:00,None,post,203776389,,21,0,0,1,235,None,vk,None,[photo],203776389,6,Putin
3,2017-09-24 19:20:47,None,post,203776389,,36,0,2,1,190,None,api,android,[photo],203776389,6,Putin
4,2017-09-27 16:40:00,None,post,203776389,,0,0,0,1,257,None,mvk,None,None,203776389,6,Putin


In [26]:
# Сохраняем итоговый результат
df_all_posts.to_csv('data_all.csv', sep=';')

Отлично! Все данные предобработаны и готовы к визуализации и моделированию. До встречи в следующем блокноте с визуализацией! 